In [23]:
import pandas as pd
import numpy as np
import itertools
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import matplotlib.pyplot as plt

In [24]:
df = pd.read_csv('dataprep.csv')

In [25]:
df.head()

Label                                          Statement
0     H  @Grumpy_P_Sloth @deanesmay feminists argue for...
1     H  1948Army of England helped the Jews to occupy ...
2     H  Dutch Leader Says Europe to collapse In 6 Week...
3     H  RT @__DeLay: The next day the Romans and the J...
4     H  RT @Bakersman_Joe: When Hitler Invited The Jew...

In [26]:
df['Label'] = df.Label.map({'H':0, 'N':1})

In [27]:
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.stem import SnowballStemmer,WordNetLemmatizer
stemmer=SnowballStemmer('english')
lemma=WordNetLemmatizer()
from string import punctuation
import re

In [28]:
def clean_review(review_col):
    review_corpus=[]
    for i in range(0,len(review_col)):
        review=str(review_col[i])
        review = re.sub(r'^@', '', review, flags=re.MULTILINE)
        review=re.sub(r'[^a-zA-Z]',' ',review)
        review = re.sub(r'^https?:\/\/.*[\r\n]*', '', review, flags=re.MULTILINE)
        review = re.sub(r'RT', '', review, flags=re.MULTILINE)
        #review=[stemmer.stem(w) for w in word_tokenize(str(review).lower())]
        review=[lemma.lemmatize(w) for w in word_tokenize(str(review).lower())]
        review=' '.join(review)
        review_corpus.append(review)
    return review_corpus

In [29]:
df['clean_review']=clean_review(df.Statement.values)
df.head(25)

Label                                          Statement  \
0       0  @Grumpy_P_Sloth @deanesmay feminists argue for...   
1       0  1948Army of England helped the Jews to occupy ...   
2       0  Dutch Leader Says Europe to collapse In 6 Week...   
3       0  RT @__DeLay: The next day the Romans and the J...   
4       0  RT @Bakersman_Joe: When Hitler Invited The Jew...   
5       0  RT @billmaher: Bravo to #Netanyahu for his 45 ...   
6       0  RT @BitchImKermit: White people literally Be l...   
7       0  RT @CharlesFinch: "The Jews were provided free...   
8       0  RT @gofran942010: "I would have killed all the...   
9       0  RT @HeymanHustle: .. @MichelleDBeadle I can do...   
10      0  RT @HSConfessionaI: "Did a PowerPoint on 9 /11...   
11      0  .@ExqCreations Who...the Jews?  They run the s...   
12      0  RT @janimine: Diversity is a code word for #Wh...   
13      0  RT @janimine: https: / /t.co /lqRf61w3CY Migra...   
14      0  RT @LadyAodh: 80% of criminals in the #Danish ...   
15      0  RT @Sanity_3: Liberal Muslim girl from Belgium...   
16      0  @_AltRight_Anew Those chimps have morphed #Sou...   
17      0  RT @YahBoyCourage: When Hitler invited the jew...   
18      0  RT @yankeebrit77: Ask yourself, why haven't yo...   
19      0  Very HITLER like !The NAZI did the same to the...   
20      0  @EqualityAlways @BrurRabbit @m_shmurda @cykell...   
21      0  @FatengOthmang if Hitler didnot kill majority ...   
22      0   @GORBtheOTHER The Jews are the masterminds idiot   
23      0  @rabite The U.S. Education is shit. They blame...   
24      0  #Trump #Trump2016 #MakeAmericaGreatAgain #Trum...   

                                         clean_review  
0   grumpy p sloth deanesmay feminist argue for ra...  
1   army of england helped the jew to occupy pales...  
2   dutch leader say europe to collapse in week th...  
3   delay the next day the roman and the jew or wa...  
4   bakersman joe when hitler invited the jew to t...  
5   billmaher bravo to netanyahu for his second gl...  
6   bitchimkermit white people literally be like i...  
7   charlesfinch the jew were provided free transp...  
8   gofran i would have killed all the jew in the ...  
9   heymanhustle michelledbeadle i can do news i c...  
10  hsconfessionai did a powerpoint on conspiracy ...  
11  exqcreations who the jew they run the slave tr...  
12  janimine diversity is a code word for whitegen...  
13  janimine http t co lqrf w cy migrant cut throa...  
14  ladyaodh of criminal in the danish capital are...  
15  sanity liberal muslim girl from belgium is exa...  
16  altright anew those chimp have morphed southaf...  
17  yahboycourage when hitler invited the jew to t...  
18  yankeebrit ask yourself why haven t you heard ...  
19  very hitler like the nazi did the same to the ...  
20  equalityalways brurrabbit m shmurda cykelly lo...  
21  fatengothmang if hitler didnot kill majority o...  
22      gorbtheother the jew are the mastermind idiot  
23  rabite the u s education is shit they blame hi...  
24  trump trump makeamericagreatagain trumptrain w...

In [30]:
def counter(review_col):
    review_corpus=[]
    for i in range(0,len(review_col)):
        num_words = 0
        review=str(review_col[i])
        words = review.split()
        for j in words:
            num_words += 1
        review_corpus.append(num_words)
    return review_corpus

In [31]:
df['counter']=counter(df.clean_review.values)
df.head()

Label                                          Statement  \
0      0  @Grumpy_P_Sloth @deanesmay feminists argue for...   
1      0  1948Army of England helped the Jews to occupy ...   
2      0  Dutch Leader Says Europe to collapse In 6 Week...   
3      0  RT @__DeLay: The next day the Romans and the J...   
4      0  RT @Bakersman_Joe: When Hitler Invited The Jew...   

                                        clean_review  counter  
0  grumpy p sloth deanesmay feminist argue for ra...       24  
1  army of england helped the jew to occupy pales...       23  
2  dutch leader say europe to collapse in week th...       21  
3  delay the next day the roman and the jew or wa...       27  
4  bakersman joe when hitler invited the jew to t...       25

In [32]:
df.drop(['Statement'],axis=1,inplace=True)
df.head()

Label                                       clean_review  counter
0      0  grumpy p sloth deanesmay feminist argue for ra...       24
1      0  army of england helped the jew to occupy pales...       23
2      0  dutch leader say europe to collapse in week th...       21
3      0  delay the next day the roman and the jew or wa...       27
4      0  bakersman joe when hitler invited the jew to t...       25

In [33]:
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['clean_review'].values)
X = tokenizer.texts_to_sequences(df['clean_review'].values)
X = pad_sequences(X)

In [62]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, SpatialDropout1D
import tensorflow as tf
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(Dropout(0.5))
model.add(LSTM(128,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model.add(LSTM(64,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model.add(Dense(2,activation='sigmoid',kernel_initializer='glorot_normal'))
model.compile(loss = tf.losses.sigmoid_cross_entropy(
    multi_class_labels,
    logits,
    weights=1.0,
    label_smoothing=0,
    scope=None,
    loss_collection=tf.GraphKeys.LOSSES,
    reduction=Reduction.SUM_BY_NONZERO_WEIGHTS), optimizer='Nadam',metrics = ['accuracy'])
print(model.summary())

NameError: name 'multi_class_labels' is not defined

In [52]:
from sklearn.cross_validation import train_test_split
y = pd.get_dummies(df['Label']).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20,random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(16288, 38)
(4072, 38)
(16288, 2)
(4072, 2)


In [53]:
X_val = X_train[:4072]
y_val = y_train[:4072]

In [54]:
partial_X_train = X_train[4072:]
partial_y_train = y_train[4072:]

In [55]:
batch_size = 64
history = model.fit(partial_X_train, 
                    partial_y_train, 
                    epochs = 10, 
                    batch_size=batch_size, 
                    validation_data=(X_val, y_val))

Train on 12216 samples, validate on 4072 samples
Epoch 1/10
12216/12216 [==============================] - 64s 5ms/step - loss: 0.0166 - acc: 0.7412 - val_loss: 2.2490e-04 - val_acc: 0.7370
Epoch 2/10
12216/12216 [==============================] - 73s 6ms/step - loss: 1.4176e-04 - acc: 0.7421 - val_loss: 8.3003e-05 - val_acc: 0.7370
Epoch 3/10
12216/12216 [==============================] - 67s 5ms/step - loss: 6.1771e-05 - acc: 0.7421 - val_loss: 4.4206e-05 - val_acc: 0.7370
Epoch 4/10
12216/12216 [==============================] - 60s 5ms/step - loss: 3.5512e-05 - acc: 0.7421 - val_loss: 2.7504e-05 - val_acc: 0.7370
Epoch 5/10
12216/12216 [==============================] - 47s 4ms/step - loss: 2.3080e-05 - acc: 0.7421 - val_loss: 1.8717e-05 - val_acc: 0.7370
Epoch 6/10
12216/12216 [==============================] - 56s 5ms/step - loss: 1.6080e-05 - acc: 0.7421 - val_loss: 1.3417e-05 - val_acc: 0.7370
Epoch 7/10
12216/12216 [==============================] - 59s 5ms/step - loss: 1.1753

In [56]:
loss, accuracy = model.evaluate(X_train,y_train , verbose=False)
print("Training Accuracy: {:.2f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.2f}".format(accuracy))

Training Accuracy: 0.74
Testing Accuracy:  0.74


In [38]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
city_labels = encoder.fit_transform(df['Label'])
city_labels

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [43]:
from keras.models import Sequential
from keras import layers

embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(input_dim=20360, 
                           output_dim=embedding_dim, 
                           input_length=100))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 50)           1018000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 5000)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                50010     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 1,068,021
Trainable params: 1,068,021
Non-trainable params: 0
_________________________________________________________________


In [44]:
history = model.fit(X_train, y_train,
                    epochs=20,
                    verbose=False,
                    validation_data=(X_test, y_test),
                    batch_size=10)
loss, accuracy = model.evaluate(X_train_dtm, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test_dtm, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

ValueError: Error when checking input: expected embedding_3_input to have shape (100,) but got array with shape (38,)

In [14]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plot_history(history)

In [15]:
from keras.models import Sequential
from keras import layers

embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 50)           63900     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                510       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 64,421
Trainable params: 64,421
Non-trainable params: 0
_________________________________________________________________


In [16]:
history = model.fit(X_train_dtm, y_train,
                    epochs=50,
                    verbose=False,
                    validation_data=(X_test_dtm, y_test),
                    batch_size=10)
loss, accuracy = model.evaluate(X_train_dtm, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test_dtm, y_test, verbose=False)
print("Testing Accuracy:{:.4f}".format(accuracy))

Training Accuracy: 0.9462
Testing Accuracy:0.7527


In [17]:
import numpy as np

def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [18]:
embedding_dim = 50
embedding_matrix = create_embedding_matrix('dataset.csv', tokenizer.word_index, embedding_dim)

In [19]:
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
nonzero_elements / vocab_size

0.0

In [21]:
print(vocab_size)

1278


In [ ]:
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))

model.add(LSTM(256,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model.add(LSTM(128,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))

In [ ]:
import numpy as np 
import pandas as pd 
import nltk
import os
import gc
from keras.preprocessing import sequence,text
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense,Dropout,Embedding,LSTM,Conv1D,GlobalMaxPooling1D
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
#pd.set_option('display.max_colwidth',100)
pd.set_option('display.max_colwidth', -1)